# Matheuristic Approach
To run, please change path in the first cell and run the second cell.

In [ ]:
path = "/Group Project/IRP_Large_Instances/IRP_High/L_abs1n50_4_H.dat"

In [ ]:
### LIBRARY
import os
from ivrp_matheuristics import *
from docplex.mp.model import Model

from dataclasses import replace
import numpy as np
import numpy.random as rnd
import networkx as nx
import matplotlib.pyplot as plt
from pathlib import Path

import sys
sys.path.append('./ALNS_Matheuristics')
from alns import ALNS, State
from alns.criteria import HillClimbing, SimulatedAnnealing, RecordToRecordTravel

### LOAD DATA
parsed = Parser(path)
ivrp = IVRP(parsed.name, parsed.depot, parsed.customers, parsed.vehicles, parsed.nPeriods, parsed.nVehicles)

### Destroy operators ###
def random_destroy(current:IVRP, random_state):
    '''
    This operator randomly selects one period and removes one customer
    from it. It is repeated p times. This movement is useful for refining the
    solution, since it does not change it much when p is small (which happens
    frequently), but still yields a major transformation when p is large.
    
    Args:
        current::IVRP
            an IVRP object before destroying
            contains IVRP.destruction attribute
        random_state::numpy.random.RandomState
            a random state specified by the random seed
    Returns:
        destroyed::EVRP
            the evrp object after destroying
    '''
    destroyed = current.copy()
    p = int(destroyed.destruction * destroyed.nPeriods + 1)
    
    # checking for each customer that is visited already
    customers_visited = [i for i, j in destroyed.y_visited.items() if j == 1]
    
    # for p times, if the number of customers visited is > 0
    for times in range(p):    
        if len(customers_visited) > 0:
            # random customer is selected
            rnd_choice = np.random.choice(range(len(customers_visited)), 1)[0]
            i, k, t = customers_visited[rnd_choice]
            customers_visited.pop(rnd_choice)
            
            # and removed from routes
            if len(destroyed.customer_visited[t-1][k]) > 2:
                destroyed.customer_visited[t-1][k].pop([node.id for node in destroyed.customer_visited[t-1][k]].index([cus.id for cus in destroyed.customers][i - 1]))
    return destroyed

def worst_destroy(current:IVRP, random_state):
    '''
    This operator goes through all customers in existing routes and selects 
    the customer that results in the highest distance travelled between neighbouring
    customers / depot.
    
    Args:
        current::IVRP
            an IVRP object before destroying
            contains IVRP.destruction attribute
        random_state::numpy.random.RandomState
            a random state specified by the random seed
    Returns:
        destroyed::EVRP
            the evrp object after destroying
    '''
    destroyed = current.copy()
    
    # initialization of variables
    customer_ids = [destroyed.depot.id] + [i.id for i in destroyed.customers]
    worst_distance = 0
    to_destroy = 0
    
    # for each period, vehicle and customer visited
    for t_period in range(1, destroyed.nPeriods + 1):
        for t_veh in range(destroyed.nVehicles):
            for node in range(1, len(destroyed.customer_visited[t_period-1][t_veh]) - 1):
                # evaluation of travel distance
                travelled_distance = round(cdist([[destroyed.customer_visited[t_period-1][t_veh][node].x, 
                                                    destroyed.customer_visited[t_period-1][t_veh][node].y]], 
                                                [[destroyed.customer_visited[t_period-1][t_veh][node+1].x, 
                                                  destroyed.customer_visited[t_period-1][t_veh][node+1].y]], 
                                                'euclidean')[0][0]) + \
                                     round(cdist([[destroyed.customer_visited[t_period-1][t_veh][node-1].x, 
                                                    destroyed.customer_visited[t_period-1][t_veh][node-1].y]], 
                                                [[destroyed.customer_visited[t_period-1][t_veh][node].x, 
                                                  destroyed.customer_visited[t_period-1][t_veh][node].y]], 
                                                'euclidean')[0][0])
                
                # worst travel distance is recorded
                if travelled_distance > worst_distance:
                    worst_distance = travelled_distance
                    to_destroy = (node, t_period-1, t_veh)
    
    # worst customer is removed from existing routes
    if to_destroy != 0:
        destroyed.customer_visited[to_destroy[1]][to_destroy[2]].pop(to_destroy[0])
    return destroyed

def consecutive_destroy(current:IVRP, random_state):
    '''
    This operator checks each customer visited whether they are visited during 
    consecutive periods. This method was used based on observations that good solutions do
    not contain consecutive visits to the same customer.
    
    Args:
        current::IVRP
            an IVRP object before destroying
            contains IVRP.destruction attribute
        random_state::numpy.random.RandomState
            a random state specified by the random seed
    Returns:
        destroyed::EVRP
            the evrp object after destroying
    '''
    destroyed = current.copy()
    
    # customer ids are taken
    customer_ids = [destroyed.depot.id] + [i.id for i in destroyed.customers]
    to_destroy = []
    
    # for each customer, check if they have been consecutively visited
    for i in range(len(destroyed.customers)):
        for t in range(destroyed.nPeriods, 1, -1):
            # if they have been consecutively visited, customer is added to 'to_destroy' list
            if (sum([destroyed.y_visited[(i+1, k, t)] for k in range(destroyed.nVehicles)]) == 1) and \
            (sum([destroyed.y_visited[(i+1, k, t-1)] for k in range(destroyed.nVehicles)]) == 1):
                to_destroy.append((t, i + 1))
                break
    
    # for each customer in the 'to_destroy' list, we remove the latest customer
    for period, node in to_destroy:
        node_id = customer_ids[node]
        for veh in range(destroyed.nVehicles):
            for cust in range(len(destroyed.customer_visited[period - 1][veh])):
                if destroyed.customer_visited[period - 1][veh][cust].id == node_id:
                    destroyed.customer_visited[period - 1][veh].pop(cust)
                    break
    
    return destroyed

def random_veh_destroy(current:IVRP, random_state):
    '''
    This operator randomly selects one vehicle and removes all routes 
    visited by the vehicle for all periods. 
    
    Args:
        current::IVRP
            an IVRP object before destroying
            contains IVRP.destruction attribute
        random_state::numpy.random.RandomState
            a random state specified by the random seed
    Returns:
        destroyed::EVRP
            the evrp object after destroying
    '''
    destroyed = current.copy()
    
    # a random vehicle is selected
    veh = random.randint(0, destroyed.nVehicles - 1)
    
    # for each time period, and the selected vehicle, routes are emptied
    for i in range(destroyed.nPeriods):
        destroyed.customer_visited[i][veh] = [destroyed.depot, destroyed.depot]
    
    return destroyed

def random_period_destroy(current:IVRP, random_state):
    '''
    This operator randomly selects one period and removes all routes for this period. 
    
    Args:
        current::IVRP
            an IVRP object before destroying
            contains IVRP.destruction attribute
        random_state::numpy.random.RandomState
            a random state specified by the random seed
    Returns:
        destroyed::EVRP
            the evrp object after destroying
    '''
    destroyed = current.copy()
    
    # a random period is selected
    period = random.randint(0, destroyed.nPeriods - 1)
    
    # for each vehicle in the selected period, the vehicles are emptied
    destroyed.customer_visited[period] = [[destroyed.depot, destroyed.depot] for i in range(destroyed.nVehicles)]
    return destroyed

### Repair operators ###
def random_insert(destroyed:IVRP, random_state):
    ''' This operator randomly inserts p customers into the current solution.
    Speciﬁcally, it selects one random customer, one random period, and one vehicles 
    and inserts the customer into the route of that period if the customer is not
    yet routed. This movement is repeated p times.
    Args:
        destroyed::IVRP
            an IVRP object after destroying
        random_state::numpy.random.RandomState
            a random state specified by the random seed
    Returns:
        repaired::IVRP
            the evrp object after repairing
    '''
    repaired = destroyed.copy()
    p = int(repaired.destruction*repaired.nPeriods + 1)
    
    unselected = []
    # get unselected customers
    for i in range(len(repaired.customers)):
        for t in range(repaired.nPeriods, 0, -1):
            if (sum([repaired.y_visited[(i+1, k, t)] for k in range(repaired.nVehicles)]) == 0):
                unselected.append((t, i + 1))
                break
    
    # p customers are randomly selected
    if len(unselected) == 0:
        to_repair = []
    else:
        rnd_choices = rnd.choice(len(unselected), size = p)
        to_repair = [unselected[i] for i in rnd_choices]
    
    # for each customer that is selected, they will be added to customers visited list
    for period, cust in to_repair:
        rand_veh = rnd.choice(range(repaired.nVehicles))
        if len(repaired.customer_visited[period-1][rand_veh]) > 2:
            rand_idx = rnd.choice(range(1, len(repaired.customer_visited[period-1][rand_veh]) - 1))
        else:
            rand_idx = 1
        repaired.customer_visited[period-1][rand_veh].insert(rand_idx, repaired.customers[cust - 1])
    
    # x and y matrix generated and solved with MP
    x, y = repaired.get_edges()
    repaired.get_DeliverQuantities(x, y)
    return repaired

def best_insert(destroyed:IVRP, random_state):
    ''' This operator inserts p customers based on a best insert approach.
    Speciﬁcally, it generates all customers that has not been visited in each period. 
    Based on this list, a customer will be inserted based on whether their insertion will
    result in the best reduction in objective function. This movement is repeated p times.
    
    Args:
        destroyed::IVRP
            an IVRP object after destroying
        random_state::numpy.random.RandomState
            a random state specified by the random seed
    Returns:
        repaired::IVRP
            the evrp object after repairing
    '''
    repaired = destroyed.copy()
    p = int(repaired.destruction*repaired.nPeriods + 1)
    
    unselected = []
    # get unselected customers
    for i in range(len(repaired.customers)):
        for t in range(repaired.nPeriods, 0, -1):
            if (sum([repaired.y_visited[(i+1, k, t)] for k in range(repaired.nVehicles)]) == 0):
                unselected.append((t, i + 1))
                break
    
    # evaluation each customer's insertion
    if len(unselected) == 0:
        to_repair = []
    else:
        for times in range(p):
            if len(unselected) > 0:
                best = repaired.objective()
                best_sol = repaired.copy()
                best_choice = -1
                for choice in range(len(unselected)):
                    # for each choice of period and customer, we evaluate by each vehicle and 
                    # index for which insertion is best
                    period, cust = unselected[choice]
                    for veh in range(repaired.nVehicles):
                        for insertion in range(1, len(repaired.customer_visited[period-1][veh])):
                            repaired_temp = repaired.copy()
                            repaired_temp.customer_visited[period-1][veh] = repaired_temp.customer_visited[period-1][veh][:insertion] + [repaired_temp.customers[cust - 1]] + repaired_temp.customer_visited[period-1][veh][insertion:]
                            #x_temp, y_temp = repaired_temp.get_edges()
                            #repaired_temp.get_DeliverQuantities(x_temp, y_temp)
                        if repaired_temp.objective() < best:
                            best = repaired_temp.objective()
                            best_sol = repaired_temp.copy()
                            best_choice = choice
                # the choice for the best insertion is removed from the list
                if best_choice != -1:
                    unselected.pop(best_choice)
    
    # x and y matrix generated and solved with MP
    x, y = repaired.get_edges()
    repaired.get_DeliverQuantities(x, y)
    return repaired

def swap_repair(destroyed:IVRP, random_state):
    '''This operator randomly swaps p pairs of customers into the current solution.
    Speciﬁcally, it selects p pairs of random customer in any existing route, and
    swaps the customers in each route. 
    
    The operator checks whether the nodes swapped has been visited in the period
    before proceeding with the swap as well. This movement is repeated p times.
    
    Args:
        destroyed::IVRP
            an IVRP object after destroying
        random_state::numpy.random.RandomState
            a random state specified by the random seed
    Returns:
        repaired::IVRP
            the evrp object after repairing
    
    '''
    # copying destroyed ivp class and defining p
    repaired = destroyed.copy()
    p = int(repaired.destruction * repaired.nPeriods + 1)
    
    # for p times
    for i in range(p):
        # finding customers in existing routes
        choices = [(idx1, idx2, idx3) for idx1, period in enumerate(repaired.customer_visited) for idx2, veh in enumerate(period) for idx3, cust in enumerate(veh[:-1]) if len(veh) > 2 and idx3 != 0]
        
        # only if there are >1 existing customers visited
        if len(choices) > 1:
            # pairs are randomly chosen
            swap_pair = np.random.choice(len(choices), 2, replace = False)
            pair_1 = choices[swap_pair[0]]
            pair_2 = choices[swap_pair[1]]
            
            # check if both customers have been visited, swapped if not
            if sum([repaired.y_visited[(repaired.customer_visited[pair_1[0]][pair_1[1]][pair_1[2]].id, i, pair_2[0]+1)] for i in range(repaired.nVehicles)]) + \
            sum([repaired.y_visited[(repaired.customer_visited[pair_2[0]][pair_2[1]][pair_2[2]].id, i, pair_1[0]+1)] for i in range(repaired.nVehicles)]) == 0:
                repaired.customer_visited[pair_1[0]][pair_1[1]][pair_1[2]], repaired.customer_visited[pair_2[0]][pair_2[1]][pair_2[2]] = \
                repaired.customer_visited[pair_2[0]][pair_2[1]][pair_2[2]], repaired.customer_visited[pair_1[0]][pair_1[1]][pair_1[2]]
      
    # x and y matrix generated and solved with MP
    x, y = repaired.get_edges()
    repaired.get_DeliverQuantities(x, y)
    return repaired

# defining random state
seed = 606
random_state = rnd.RandomState(seed)
alns = ALNS(random_state)

# construct random initialized solution
ivrp.random_initialize(seed)
print("Initial solution objective is {}.".format(ivrp.objective()))

# adding of destroy operators to ALNS method
alns.add_destroy_operator(random_destroy)
alns.add_destroy_operator(worst_destroy)
alns.add_destroy_operator(consecutive_destroy)
alns.add_destroy_operator(random_veh_destroy)
alns.add_destroy_operator(random_period_destroy)

# adding of repair operators to ALNS method
alns.add_repair_operator(random_insert)
alns.add_repair_operator(best_insert)
alns.add_repair_operator(swap_repair)

# # run ALNS
# # select criterion
criterion = SimulatedAnnealing(5e+46, 5e+44, 5e+41)

# assigning weights to methods
omegas = [2, 1.5, 1, 0.3]
lambda_ = 0.9
result = alns.iterate(ivrp, omegas, lambda_, criterion,
                      iterations = 1000, collect_stats = True)

# result
solution = result.best_state
objective = solution.objective()
print('Best heuristic objective is {}.'.format(objective))


### generation of file output
# no. of customers
n_cust = len(solution.customers)
# no. of nodes (customers+depot)
n_node = n_cust + 1
#no. of vehicles
n_veh = len(solution.vehicles[0])
#no. of periods (incl. t=0 for initial state)
t_period = solution.nPeriods + 1

str_builder = ['Instance: {}\nObjective: {}\nTime Taken:\n'.format(solution.name, solution.objective())]
str_builder.append('Inventory Levels:-\n')
for i in range(n_node):
    inven_level_temp = []
    for j in range(t_period):
        inven_level_temp.append(round(solution.DeliverQuantities_solution.as_name_dict().get("inv_"+str(i)+"_"+str(j), 0)))
    str_builder.append('Node ' + str(i) + ': ' + ', '.join(str(k) for k in inven_level_temp))
str_builder.append('\n')

str_builder.append('Consolidated Routes:-')
quantity_delivered = []
for t in range(1, t_period):
    str_builder.append('\nTime Period ' + str(t) + ":")
    for k in range(n_veh):
        str_builder.append("Vehicle " + str(k) + ": " + ", ".join(["("+str(i.id)+", "+str(solution.DeliverQuantities_solution.as_name_dict().get('qty_'+str(i.id)+"_"+str(k)+"_"+str(t), 0))+")" for i in solution.customer_visited[t-1][k]]))

with open('{}_MP_Solution.txt'.format(solution.name), 'w') as f:
    f.write('\n'.join(str_builder))